In [2]:
import os
from cot import Collection
from cot.generate import FRAGMENTS
from rich.pretty import pprint
import json

In [2]:
# number_examples = 100
# dataset_name_splits = [("worldtree", "test"), ("open_book_qa", "test")]
# api_service_model_time = [("cohere", "command-xlarge-nightly", 0), ("huggingface_hub", "google/flan-t5-xl", 20)]

In [3]:
def get_answer_extraction(type,choices): # collection[dataset_name][split][0]["choices"]
    if type == "bool": 
        return "kojima-yes-no"
    elif type == "multiplechoice":
        n_choices = len(choices)
        if n_choices == 3: answer_extraction_key = 'kojima-A-C'
        elif n_choices == 4: answer_extraction_key = 'kojima-A-D'
        elif n_choices == 5: answer_extraction_key = 'kojima-A-E'
        elif n_choices == 6: answer_extraction_key = 'kojima-A-F'
        return(answer_extraction_key)
    else: raise ValueError("type must be bool or multiplechoice")

In [4]:
# join strings from a list with underscore
def join_strings(list_of_strings):
    joined_string = ""
    for string in list_of_strings:
        joined_string += ("_" + str(string))
    return(joined_string)

In [5]:
def model_sprint(dataset_name_splits, api_service_model, cot_trigger_keys, number_examples):
    for dataset_name, split in dataset_name_splits:
        collection = Collection([dataset_name], verbose=False)
        collection = collection.select(split=split, number_samples=number_examples)
        answer_extraction_key = get_answer_extraction(collection[dataset_name][split][0]["type"], collection[dataset_name][split][0]["choices"])
        for api_service, model, api_time_interval in api_service_model:

            config={
                "cot_trigger_keys": cot_trigger_keys,
                "answer_extraction_keys": [answer_extraction_key], # Therefore, among A through C/D/E/F, the answer is'
                "author" : "thoughtsource",
                "api_service": api_service,
                "engine": model,
                "temperature": 0,
                "max_tokens": 512,
                "api_time_interval": api_time_interval,
                "verbose": False,
                "warn": False,
            }
        
            collection.generate(config=config)
            collection.evaluate()
            collection.dump(dataset_name + "_" + split + "_" + str(number_examples) + "_" + api_service + "_" + model + join_strings(config["cot_trigger_keys"]) + ".json")

In [12]:
collection = Collection(["medmc_qa"], verbose=False, load_pregenerated_cots="all")

Loading medmc_qa...


In [13]:
collection = collection.select(split="validation")

In [5]:
# run through every item of the dataset and save the index of the items that have a generated_cot
idx = []
for i,ex in enumerate(collection["medmc_qa"]["validation"]):
    if ex["generated_cot"] != []:
        idx.append(i)

In [6]:
len(idx)

1000

In [14]:
collection["medmc_qa"]["validation"] = collection["medmc_qa"]["validation"].select(idx)

In [16]:
collection = collection.select("validation", number_samples=100)

In [17]:
collection

| Name     | Train   |   Valid | Test   |
|----------|---------|---------|--------|
| medmc_qa | -       |     100 | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [18]:
# run through every item of the dataset and save the index of the items that have a generated_cot
control_idx = []
for i,ex in enumerate(collection["medmc_qa"]["validation"]):
    if ex["generated_cot"] != []:
        control_idx.append(i)

In [19]:
len(control_idx)

100

In [8]:
collection

| Name     | Train   |   Valid | Test   |
|----------|---------|---------|--------|
| medmc_qa | -       |    4183 | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [9]:
dataset = collection["medmc_qa"]["validation"]

In [10]:
dataset_sorted = dataset.filter(lambda example: example["generated_cot"] != [])

In [11]:
dataset_sorted

Dataset({
    features: ['id', 'ref_id', 'question', 'type', 'choices', 'context', 'cot', 'answer', 'generated_cot', 'feedback'],
    num_rows: 1000
})

In [12]:
collection["medmc_qa"]["validation"] = dataset_sorted

In [13]:
collection

| Name     | Train   |   Valid | Test   |
|----------|---------|---------|--------|
| medmc_qa | -       |    1000 | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [15]:
collection.dump("medmc_qa_filtered_1000.json")

: 

: 

In [3]:
import random

In [8]:
random.seed(0)
random_ids = random.sample(range(1000), 100)
random_ids = sorted(random_ids)

In [7]:
len(random_ids)

100

In [2]:
from cot.create_thoughtsource_100 import create_medmc

In [3]:
create_medmc()

Loading medmc_qa...


In [8]:
dataset_sorted.to_json("medmc_qa_filtered.json")

: 

: 

In [7]:
collection.dump("medmc_foo_100.json")

In [30]:
coll = Collection.from_json("medmc_foo_100.json")

In [31]:
coll

| Name     | Train   |   Valid | Test   |
|----------|---------|---------|--------|
| medmc_qa | -       |     100 | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [10]:
collection = Collection.from_json("worldtree_test_100_cohere_command-xlarge-nightly.json")
collection.evaluate()

Evaluating worldtree test...


  0%|          | 0/100 [00:00<?, ?ex/s]

{'worldtree': {'test': {'accuracy': {'command-xlarge-nightly': {'None_None_kojima-A-D': 0.63,
     'None_kojima-01_kojima-A-D': 0.63,
     'None_kojima-02_kojima-A-D': 0.65,
     'None_kojima-03_kojima-A-D': 0.59}}}}}

In [12]:
collection = Collection.from_json("worldtree_test_100_openai_text-davinci-003.json")
collection.evaluate()

Evaluating worldtree test...


  0%|          | 0/100 [00:00<?, ?ex/s]

{'worldtree': {'test': {'accuracy': {'text-davinci-003': {'None_kojima-01_kojima-A-D': 0.73}}}}}

In [13]:
collection = Collection.from_json("open_book_qa_test_100_cohere_command-xlarge-nightly.json")
collection.evaluate()

Evaluating open_book_qa test...


  0%|          | 0/100 [00:00<?, ?ex/s]

{'open_book_qa': {'test': {'accuracy': {'command-xlarge-nightly': {'None_kojima-01_kojima-A-D': 0.38}}}}}

In [14]:
collection = Collection.from_json("open_book_qa_test_100_openai_text-davinci-003.json")
collection.evaluate()

Evaluating open_book_qa test...


  0%|          | 0/100 [00:00<?, ?ex/s]

{'open_book_qa': {'test': {'accuracy': {'text-davinci-003': {'None_kojima-01_kojima-A-D': 0.68}}}}}

In [ ]:
# one example with one prompt took 1.5 minutes
# so 100 examples with 4 prompts probably take 1.5 * 100 * 4 = 600 minutes = 10 hours
# started at 19h10

In [ ]:
collection = Collection([dataset_name], verbose=False)
collection = collection.select(split=split, number_samples=number_examples)
print(collection)

Loading commonsense_qa...
| Name           | Train   |   Valid | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       |       1 | -      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']


In [ ]:
# Configuration of the input and parameters of the language model 
config={
    "cot_trigger_keys": [
        None,         # no cot-trigger
        'kojima-01',  # Answer: Let's think step by step.
        'kojima-02',  # Answer: First,
        'kojima-03'], # Answer: Let's think about this logically.
    "answer_extraction_keys": [answer_extraction_key], # Therefore, among A through C/D/E/F, the answer is'
    "author" : "thoughtsource",
    "api_service": api_service,
    "engine": model,
    "temperature": 0,
    "max_tokens": 512,
    "verbose": False,
    "warn": False,
}

In [ ]:
# Generating chains-of-thought and answer extractions
collection.generate(config=config)


        You are about to  call an external API  in total 8 times, which  may produce costs .
        API calls for reasoning chain generation: 1 samples  * 1 instructions  * 4 reasoning chain triggers
        API calls for answer extraction: n_samples  1 samples  * 1 instructions  * 4 reasoning chain triggers * 1 answer extraction triggers
        Do you want to continue? y/n
        


In [ ]:
collection.evaluate()

Evaluating commonsense_qa validation...


{'commonsense_qa': {'validation': {'accuracy': {'google/flan-t5-xl': {'None_None_kojima-A-E': 1.0,
     'None_kojima-01_kojima-A-E': 1.0,
     'None_kojima-02_kojima-A-E': 1.0,
     'None_kojima-03_kojima-A-E': 1.0}}}}}

In [ ]:
collection.dump(dataset_name + "_" + split + "_" + str(number_examples) + ".json")

In [13]:
dataset_name + "_" + split + "_" + str(number_examples)

'commonsense_qa_validation_1'

In [7]:
# Randomly select 10 rows from train split
med = Collection(["med_qa"], verbose=False, load_pregenerated_cots="all")
med_test_1 = med.select(split="test", number_samples=1)
med_test_1

Loading med_qa...


| Name   | Train   | Valid   |   Test |
|--------|---------|---------|--------|
| med_qa | -       | -       |      1 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [8]:
med_test_1.dump("med_test_1.json")

In [13]:
# Note that you can also sample from multiple datasets into one collection:
collection_medical = Collection(["med_qa", "pubmed_qa"], verbose=False)
collection_medical_100 = collection_medical.select(split="train", number_samples=100)
collection_medical_100

Loading med_qa...
Loading pubmed_qa...


| Name      |   Train | Valid   | Test   |
|-----------|---------|---------|--------|
| med_qa    |     100 | -       | -      |
| pubmed_qa |     100 | -       | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'medmc_qa', 'open_book_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

## 2. Generating reasoning chains and extracting answers

In [ ]:
class creator(object):
    pass

In [ ]:
name_subset = [("commonsense_qa", "validation")]

In [ ]:
commonsense = creator()

In [ ]:
for name, value in (name_subset):
    setattr(commonsense, name, value)

In [ ]:
commonsense.__dict__

{'commonsense_qa': 'validation'}

#### Using predefined text snippets

ThoughtSource comes pre-loaded with a large [collection of text snippets ('prompt fragments')](https://github.com/OpenBioLink/ThoughtSource/blob/main/libs/cot/cot/fragments.json) to elicit chain-of-thought reasoning in large language models and to extract answers from chains-of-thought. Let's see how prompt fragments look like:

In [5]:
# Chain of thought prompts
pprint(list(FRAGMENTS["cot_triggers"].items()))

[
│   ('kojima-01', "Answer: Let's think step by step."),
│   ('kojima-02', 'Answer: We should think about this step by step.'),
│   ('kojima-03', 'Answer: First,'),
│   ('kojima-04', 'Answer: Before we dive into the answer,'),
│   ('kojima-05', 'Answer: Proof followed by the answer.'),
│   ('kojima-06', "Answer: Let's think step by step in a realistic way."),
│   ('kojima-07', "Answer: Let's think step by step using common sense and knowledge."),
│   ('kojima-08', "Answer: Let's think like a detective step by step."),
│   ('kojima-09', "Answer: Let's think about this logically."),
│   ('kojima-10', "Answer: Let's think step by step. First,"),
│   ('kojima-11', "Answer: Let's think"),
│   ('kojima-12', "Answer: Let's solve this problem by splitting it into steps."),
│   ('kojima-13', 'Answer: The answer is after the proof.'),
│   ('kojima-14', "Answer: Let's be realistic and think step by step."),
│   ('lievin-01', "Answer: Let's derive the differential diagnosis step by step."),
│   (
│   │   'lievin-02',
│   │   "Answer: Let's use step by step inductive reasoning, given the medical nature of the question."
│   ),
│   ('lievin-03', "Answer: Let's differentiate using step by step reasoning like a medical expert."),
│   ('lievin-04', "Answer: Let's think step by step using deductive reasoning."),
│   ('lievin-05', "Answer: Let's differentiate using step by step reasoning ."),
│   ('lievin-06', "Answer: Let's think step by step to arrive at one of the options."),
│   ('lievin-07', "Answer: Let's break the problem into multiple steps."),
│   (
│   │   'lievin-08',
│   │   "Answer: Let's use step by step deductive reasoning, given the medical nature of the question."
│   ),
│   ('lievin-09', "Answer: Let's think step by step like a doctor."),
│   ('lievin-10', "Answer: Let's think step by step like a medical expert."),
│   ('lievin-11', "Answer: Let's summarize the facts step by step."),
│   ('lievin-12', "Answer: Let's think step by step using inductive reasoning."),
│   ('lievin-13', "Answer: Let's think step by step using deductive reasoning like a medical expert."),
│   ('lievin-14', "Answer: Let's be concise and think step by step."),
│   ('lievin-15', "Answer: Let's differentiate using step by step deductive reasoning like a medical expert."),
│   ('lievin-16', "Answer: Let's argue step by step."),
│   ('lievin-17', "Answer: Let's think step by step like a clinician."),
│   ('lievin-18', "Answer: Let's reflect on each answer option step by step."),
│   ('lievin-19', "Answer: Let's reason and differentiate options step by step like a medical expert."),
│   ('lievin-20', "Answer: Let's differentiate using step by step inductive reasoning like a medical expert."),
│   ('lievin-21', "Answer: Let's think step by step given every option equal consideration."),
│   ('lievin-22', "Answer: Let's think step by step like a scientist."),
│   ('lievin-23', "Answer: Let's use step by step inductive reasoning."),
│   ('lievin-24', "Answer: Let's work by elimination step by step."),
│   ('lievin-25', "Answer: Let's use step by step deductive reasoning."),
│   ('lievin-26', "Answer: Let's follow a Bayesian step by step approach."),
│   ('lievin-27', "Answer: Let's reflect on each option from the least likely to the most likely."),
│   ('lievin-28', "Answer: Let's use step by step Bayesian reasoning, given the medical nature of the question.")
]

In [7]:
# Answer extraction prompts
pprint(list(FRAGMENTS["answer_extractions"].items()))

[
│   ('kojima-01', 'Therefore, the answer is'),
│   ('kojima-02', 'Therefore,'),
│   ('kojima-03', 'The answer is'),
│   ('kojima-numerals', 'Therefore, the answer (arabic numerals) is'),
│   ('kojima-yes-no', 'Therefore, the answer (Yes or No) is'),
│   ('kojima-A-C', 'Therefore, among A through C, the answer is'),
│   ('kojima-A-D', 'Therefore, among A through D, the answer is'),
│   ('kojima-A-E', 'Therefore, among A through E, the answer is'),
│   ('kojima-A-F', 'Therefore, among A through F, the answer is')
]

#### Configuration parameters

In [2]:
# overview of all available configurations
from cot.config import Config as config_overview
print(f'\033[94m {config_overview.__doc__[48:]}')

 
    "instruction_keys": list(str) - Determines which instruction_keys are used from fragments.json,
        the corresponding string will be inserted under "instruction" in the fragments. Default: [None] (No instruction)
    "cot_trigger_keys": list(str) - Determines which cot triggers are used from fragments.json,
        the corresponding string will be inserted under "cot_trigger" in the fragments. Default: ["kojima-01"]
    "answer_extraction_keys": list(str) - Determines which answer extraction prompts are used from fragments.json,
        the corresponding string will be inserted under "answer" in the fragments. Default: ["kojima-01"]
    "template_cot_generation": string - is the model input in the text generation step, variables in brackets.
        Only variables of this list are allowed: "instruction", 'question", "answer_choices", "cot_trigger"
        Default: {instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}
    "template_answer_extraction": string - is the 

#### Defining your own template (optional)

The default chain of thought generation template as shown above is: "{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}". </br>
You could also define your own template with a different structure and even free text:

In [17]:
# print("Answer this question:\n{question}\n{answer_choices}\n\nGive a detailed explanation of your answer.")

If you define your custom chain of thought generation template, do not forget to also define a custom answer extraction template.

In [18]:
# print("Answer this question:\n{question}\n{answer_choices}\n\nGive a detailed explanation of your answer.{cot}\n{answer_extraction}")

### 2.1 Using a Mock-API to create reasoning chains and extract answers

In [19]:
# Configuration of the input and parameters of the language model 
config={
    # We compare three different prompts for the chain of thought generation:
    # "Answer: Let's think step by step." and 'Answer: We should think about this step by step.', and "Answer: First," 
    "cot_trigger_keys": ['kojima-01','kojima-02', 'kojima-03'],

    # We use the same answer extraction prompt for all three prompts
    # "Therefore, among A through D, the answer is"
    "answer_extraction_keys": ['kojima-A-D'], 
    
    "author" : "your_name",
    "api_service": "mock_api", # <--- We use a mock API here for demonstration purposes of the tutorial.
    "engine": "", 
    "temperature": 0,
    "max_tokens": 512,
    "verbose": False,
    "warn": True,
}

#### Generating reasoning chains and extracting answers in one call

In [20]:
# Generating chains-of-thought and answer extractions
worldtree_10.generate(config=config)

As above was a fake call to the mock API we now **load a prepared dataset** with real model answers for the purpose of the tutorial:

In [21]:
# loading a pre-generated example for the purpose of this tutorial
worldtree_10 = Collection.from_json("worldtree_10.json")

### 2.2 Using your own API to create reasoning chains and extract answers

ThoughtSource can connect to external AI service providers such as the [OpenAI API](https://openai.com/api/) or the [Hugging Face Hub](https://huggingface.co/docs/hub/index). Set your token, 'api_service' and 'engine' parameters accordingly.

In this tutorial we will use the Hugging Face Hub, which is for free. You can make an account and then copy your token from the Hugging Face settings page.

To use the API you need to set the environment variable `HUGGINGFACEHUB_API_TOKEN` to your API token:

In [22]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "<token>"   # <--- set token (can be found in your Hugging Face settings page)
# os.environ["OPENAI_API_KEY"] = "<token>"  # <--- Set token for which API you want to use

In [23]:
# Configuration of the input and parameters of the language model 
config={
    # We compare three different prompts for the chain of thought generation:
    # "Answer: Let's think step by step." and 'Answer: We should think about this step by step.', and "Answer: First," 
    "cot_trigger_keys": ['kojima-01','kojima-02', 'kojima-03'],

    # We use the same answer extraction prompt for all three prompts
    # "Therefore, among A through D, the answer is"
    "answer_extraction_keys": ['kojima-A-D'], 
    
    "api_service": "huggingface_hub", # <--- Select which API you want to use
    "engine": "google/flan-t5-xl", # <--- Select which model you want to use
    "temperature": 0,
    "max_tokens": 512,
    "verbose": False,
    "warn": True,
}

In [24]:
# Loading just one sample from the dataset so it runs faster
worldtree = Collection(["worldtree"], verbose=False)
worldtree_1 = worldtree.select(split="train", number_samples=1) # just selecting 1 sample, change that if you want to run on more

Loading worldtree...


This is a **two step process:**
 - The language model first answers a question with a detailed reasoning chain.
 - The language model then extracts the answer from its own reasoning chain.

The code does it automatically at once, but it helpful to have in mind the underlying two step process. </br>
Therefore you need two API calls for each example as shown in the warning.

In [17]:
# Calling the external API will approximately take 30 seconds for this example
worldtree_1.generate(config=config)


        You are about to  call an external API  in total 6 times, which  may produce costs .
        API calls for reasoning chain generation: 1 samples  * 1 instructions  * 3 reasoning chain triggers
        API calls for answer extraction: n_samples  1 samples  * 1 instructions  * 3 reasoning chain triggers * 1 answer extraction triggers 
        Do you want to continue? y/n
        


If your cannot press 'y' because your coding environment is not interactive, set "warn" to false in the config to deactivate the warning.

## 3. Evaluation of model answers and downloading all results

In [18]:
worldtree_10.evaluate()

Evaluating worldtree train...


{'worldtree': {'train': {'accuracy': {'text-davinci-003': {'None_kojima-01_kojima-A-D': 0.7,
     'None_kojima-02_kojima-A-D': 0.7,
     'None_kojima-03_kojima-A-D': 0.8}}}}}

Save the file, which now includes the dataset and your generated reasoning chains, extracted answers and evaluation results.

In [19]:
# Save the file that includes all generated chains of thought and answer extractions a the evaluation results
worldtree_10.dump("worldtree_10.json")

If you used your own API, use this code to evaluate and save the results:

In [20]:
# print(worldtree_1.evaluate())
# worldtree_1.dump("worldtree_1.json")

## 4. Inspect the model outputs in the Web Tool

### Here is the link: **[ThoughtSource Annotator](http://thought.samwald.info:3000/)**

Just **upload your just downloaded 'worldtree_10.json' file** to the web tool.